In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.datasets import mnist
from tensorflow.contrib.layers import batch_norm, fully_connected


def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:

reset_graph()
'''Preparing===================================================================='''
(X_train, y_train), (X_test, y_test) = mnist.load_data()
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X_train1 = X_train[:3000]
y_train1 = y_train[:3000].astype(np.int32)
X_test1 = X_test[:1000]
y_test1 = y_test[:1000].astype(np.int32)

'''Using SVD to rotate image===================================================='''
sigma = 1
def svd(all_images):
    new_all_images = []
    for image in all_images:
        U, S, VT = np.linalg.svd(image, full_matrices=0)
        N = 28
        cutoff = (4 / np.sqrt(3)) * np.sqrt(N) * sigma
        r = np.min(np.where(S > cutoff))
        
        new_image = U[:, :(r+1)] @ np.diag(S[ :(r+1)]) @ VT[ :(r+1), :]
        new_all_images.append(new_image)
    return new_all_images
'''============================================================================='''

X_train = np.array(svd(X_train1)).astype(np.float32) / 255.0
X_test = np.array(svd(X_test1)).astype(np.float32) / 255.0

X_train = X_train.reshape(3000, 28*28)
X_test = X_test.reshape(1000, 28*28)

# def one_hot(inputs):
#     one_hot_encoded = np.zeros((len(inputs), 10))
#     for i, j in enumerate(inputs):
#         one_hot_encoded[i][j] = 1
#     return one_hot_encoded

# y_train = one_hot(y_train)
# y_test = one_hot(y_test)

m_train, n_train = X_train.shape

n_epochs = 5
batch_size = 50
n_batches = int(np.ceil(m_train/batch_size))

config = tf.ConfigProto()
config.log_device_placement = True

def fetch_batch(epoch, batch_index):
    np.random.seed(epoch*n_batches + batch_index)
    indices = np.random.randint(m_train, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_train, y_train

'''placeholder=================================================================='''
with tf.device(tf.train.replica_device_setter(ps_tasks=2,
                                              ps_device='/job:ps',
                                              worker_device='/job:worker')):
    X = tf.placeholder(dtype=tf.float32, shape=(None, 28*28), name='X')
    y = tf.placeholder(dtype=tf.int32, shape=(None), name='y')

    
is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
bn_params = {
 'is_training': is_training,
 'decay': 0.99,
 'updates_collections': None
}

with tf.device('/job:ps/task:0'):
    '''hidden layer=================================================================================================='''
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1", normalizer_fn=batch_norm, normalizer_params=bn_params)

    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2", normalizer_fn=batch_norm, normalizer_params=bn_params)

    logits = fully_connected(hidden2, n_outputs, activation_fn=None,scope="outputs", normalizer_fn=batch_norm, normalizer_params=bn_params)

with tf.device('/job:worker/task:0'):
    '''Loss====================================================================='''
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

'''optimizer===================================================================='''
def learning_schedule(t):
    return 1 / (100 * (t+1))

t = tf.placeholder(dtype=tf.float32, shape=(None))

with tf.device('/job:worker/task:1'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_schedule(t))
    training_op = optimizer.minimize(loss)

'''accuracy====================================================================='''
with tf.device('/job:worker/task:2'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.Session('grpc://127.0.0.1:2221', config=config) as sess:

    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index)
            _accuracy, _loss, _ = sess.run([accuracy, loss, training_op], feed_dict={X: X_batch, y: y_batch, t: epoch})
        _test_accuracy = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
        print("\t\tEpoch: {} \ttLoss: {}, \tTrain_Accuracy: {} \tTest_Accuracy: {}".format(epoch, _loss, _accuracy*100, _test_accuracy*100))

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Please use `layer.__call__` method instead.
